In [1]:
import pandas as pd

In [2]:
path = "2024-08.csv"
df = pd.read_csv(path)

ciclo_estaciones_path = "ciclo_estaciones_cluster.csv.gz"

df_ciclo_estaciones =pd.read_csv(ciclo_estaciones_path, compression='gzip')

In [3]:
df.shape

(1891319, 9)

## Clean

In [4]:
'''
Adding the Fecha_Hora_Retiro and Fecha_Hora_Arribo columns to the dataframe
'''
def add_fecha_hora(df):
  # Validate 'Fecha_Hora_Retiro' | 'Fecha_Hora_Arribo' columns
  if 'Fecha_Hora_Retiro' in df.columns and 'Fecha_Hora_Arribo' in df.columns:
    return df

  df['Fecha_Hora_Retiro'] = pd.to_datetime(
    df['Fecha_Retiro'] + ' ' + df['Hora_Retiro'], format='%d/%m/%Y %H:%M:%S'
  )
  df['Fecha_Hora_Arribo'] = pd.to_datetime(
    df['Fecha_Arribo'] + ' ' + df['Hora_Arribo'], format='%d/%m/%Y %H:%M:%S'
  )
  return df

'''
Adding the Duracion_Viaje and Duracion_Viaje_Minutos columns to the dataframe
'''
def add_duracion_viaje(df):
  # Validate 'Duracion_Viaje' | 'Duracion_Viaje_Minutos' columns
  if 'Duracion_Viaje' in df.columns and 'Duracion_Viaje_Minutos' in df.columns:
    return df
  df['Duracion_Viaje'] = df['Fecha_Hora_Arribo'] - df['Fecha_Hora_Retiro']
  df['Duracion_Viaje_Minutos'] = df['Duracion_Viaje'].dt.total_seconds() / 60
  return df

'''
Replacing '?' with 'O' in the Genero_Usuario column
'''
def replace_genero_usuario(df):
  df['Genero_Usuario'] = df['Genero_Usuario'].replace('?', 'O')
  return df

'''
Validating date start and date end
'''
def validate_date_start_end(start, end, df):
  df_valid_dates = df[(df['Fecha_Hora_Retiro'] >= start) & 
        (df['Fecha_Hora_Retiro'] <= end) &
        (df['Fecha_Hora_Arribo'] >= start) & 
        (df['Fecha_Hora_Arribo'] <= end)]
  return df_valid_dates


'''
Dropping columns that are not needed
'''
def drop_columns(df):
  columns=[
    'Bici',
    'Fecha_Retiro',
    'Hora_Retiro',
    'Fecha_Arribo',
    'Hora_Arribo',
    'Duracion_Viaje',
    'Retiro_Valido',
    'Arribo_Valido'
  ]

  df_droped = df.drop(
    columns=[col for col in columns if col in df.columns],
  )

  return df_droped

'''
Renaming columns
'''
def rename_columns(df):
  df.rename(
    columns={
      'Ciclo_EstacionArribo': 'Ciclo_Estacion_Arribo',
    },
    inplace=True
  )
  return df

'''
Adding Grupo_Edad column
'''
def add_grupo_edad(df):
    bins = [0, 18, 24, 35, 45, 55, 65, 75, 85, 95, 105]
    labels = ['-18', '18-23', '24-34', '35-44', '45-54', '55-64', '65-74', '75-84', '85-94', '95+']
    df['Grupo_Edad'] = pd.cut(df['Edad_Usuario'], bins=bins, labels=labels, right=False)
    return df

'''
Set from valid stations
'''
valid_stations = set(df_ciclo_estaciones['short_name'].astype(str))

'''
Validate station
'''
def valid_station(station):
    return str(station) in valid_stations

'''
Adding Retiro_Valido Arribo_Valido columns
'''
def add_valid_trip(df):
    df['Retiro_Valido'] = df['Ciclo_Estacion_Retiro'].apply(valid_station)
    df['Arribo_Valido'] = df['Ciclo_Estacion_Arribo'].apply(valid_station)
    return df

'''
Filter only by Retiro_Valido & Arribo_Valido
'''
def filter_valid_trip(df):
    df = df[df['Retiro_Valido'] & df['Arribo_Valido']]
    return df

'''
Drop null
'''
def drop_null(df):
    df_droped = df.dropna()
    return df_droped

'''
Validate types
'''
def validate_df_types(df):
    df_valid_types = df.astype({
        'Genero_Usuario': 'category',
        'Edad_Usuario': 'int',
        'Ciclo_Estacion_Retiro': 'category',
        'Ciclo_Estacion_Arribo': 'category',
        'Cluster_Retiro': 'category',
        'Cluster_Arribo': 'category'
    })
    return df_valid_types

'''
Add Cluster_Retiro & Cluster_Retiro columns
'''

def add_cluster(df):
    # Cluster_Retiro
    df_trips_cluster_retiro = pd.merge(
        df,
        df_ciclo_estaciones[['short_name', 'cluster']],
        left_on='Ciclo_Estacion_Retiro',
        right_on='short_name',
        how='left'
    )
    
    df_trips_cluster_retiro = df_trips_cluster_retiro.rename(
        columns={'cluster': 'Cluster_Retiro'}
    )
    
    df_trips_cluster_retiro = df_trips_cluster_retiro.drop('short_name', axis=1)
    
    # Cluster_Arribo
    
    df_trips_valid_clusters = pd.merge(
        df_trips_cluster_retiro,
        df_ciclo_estaciones[['short_name', 'cluster']],
        left_on='Ciclo_Estacion_Arribo',
        right_on='short_name',
        how='left'
    )
    
    df_trips_valid_clusters = df_trips_valid_clusters.rename(
        columns={'cluster': 'Cluster_Arribo'}
    )
    
    df_trips_valid_clusters = df_trips_valid_clusters.drop('short_name', axis=1)
    
    return df_trips_valid_clusters

In [5]:
'''
Cleaning dataframe
'''
def clean_df(df):
    august_start_date = '2024-08-01'
    august_end_date = '2024-08-31 23:59:59'

    df_cleaned = rename_columns(df)
    df_cleaned = replace_genero_usuario(df_cleaned)
    df_cleaned = add_valid_trip(df_cleaned)
    df_cleaned = add_grupo_edad(df_cleaned)
    df_cleaned = add_fecha_hora(df_cleaned)
    df_cleaned = add_duracion_viaje(df_cleaned)
    df_cleaned = validate_date_start_end(
        august_start_date,
        august_end_date,
        df_cleaned
    )

    df_cleaned = filter_valid_trip(df_cleaned)
    df_cleaned = add_cluster(df_cleaned)
    df_cleaned = drop_columns(df_cleaned)

    df_cleaned = drop_null(df_cleaned)
    df_cleaned = validate_df_types(df_cleaned)
    
    return df_cleaned



In [6]:
df_viajes = clean_df(df)

In [7]:
df_viajes.isnull().sum()

Genero_Usuario            0
Edad_Usuario              0
Ciclo_Estacion_Retiro     0
Ciclo_Estacion_Arribo     0
Grupo_Edad                0
Fecha_Hora_Retiro         0
Fecha_Hora_Arribo         0
Duracion_Viaje_Minutos    0
Cluster_Retiro            0
Cluster_Arribo            0
dtype: int64

In [8]:
len(df_viajes)

1891041

## Subsets

In [9]:
month_general_stats = {
    "Total_Viajes": [len(df_viajes)],
    "Total_Ciclo_Estaciones": [len(df_ciclo_estaciones)],
    "Duracion_Promedio_Viaje_Minutos": [df_viajes['Duracion_Viaje_Minutos'].mean()]
}

df_month_general_stats = pd.DataFrame(month_general_stats)
df_month_general_stats
# Export csv
df_month_general_stats.to_csv('agosto-resume.csv.gz', index=False, compression='gzip')
df_viajes.to_csv('agosto-trips.csv.gz', index=False, compression='gzip')


In [10]:
df_viajes.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1891041 entries, 0 to 1891061
Data columns (total 10 columns):
 #   Column                  Dtype         
---  ------                  -----         
 0   Genero_Usuario          category      
 1   Edad_Usuario            int64         
 2   Ciclo_Estacion_Retiro   category      
 3   Ciclo_Estacion_Arribo   category      
 4   Grupo_Edad              category      
 5   Fecha_Hora_Retiro       datetime64[ns]
 6   Fecha_Hora_Arribo       datetime64[ns]
 7   Duracion_Viaje_Minutos  float64       
 8   Cluster_Retiro          category      
 9   Cluster_Arribo          category      
dtypes: category(6), datetime64[ns](2), float64(1), int64(1)
memory usage: 86.6 MB


In [11]:
df_viajes

,Genero_Usuario,Edad_Usuario,Ciclo_Estacion_Retiro,Ciclo_Estacion_Arribo,Grupo_Edad,Fecha_Hora_Retiro,Fecha_Hora_Arribo,Duracion_Viaje_Minutos,Cluster_Retiro,Cluster_Arribo
0,O,53,331,312,45-54,2024-08-01 00:01:50,2024-08-01 00:04:26,2.600000,3,3
1,F,20,104,055,18-23,2024-08-01 00:00:06,2024-08-01 00:05:00,4.900000,4,4
2,M,32,008,026,24-34,2024-08-01 00:00:47,2024-08-01 00:05:22,4.583333,4,4
3,M,32,077,068,24-34,2024-08-01 00:02:05,2024-08-01 00:05:27,3.366667,0,0
4,M,37,054,154,35-44,2024-08-01 00:01:28,2024-08-01 00:05:52,4.400000,0,0
...,...,...,...,...,...,...,...,...,...,...
1891057,M,38,013,089,35-44,2024-08-31 23:36:24,2024-08-31 23:59:40,23.266667,0,4
1891058,M,33,014,103,24-34,2024-08-31 23:46:40,2024-08-31 23:59:41,13.016667,0,1
1891059,M,26,432,357,24-34,2024-08-31 23:46:13,2024-08-31 23:59:44,13.516667,2,3
1891060,F,33,029,504,24-34,2024-08-31 23:38:30,2024-08-31 23:59:47,21.283333,4,1
